In [13]:
# Import Dependencies
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import os

In [15]:
# Define paths
#train_dir = 'ASL_Alphabet_Dataset/asl_alphabet_train'
#test_dir = 'ASL_Alphabet_Dataset/asl_alphabet_test'
"""
Uncomment the above lines.
Create your own personal directories for the training and testing data.
Using the same variable names
"""
train_dir = 'consolidated_asl_alphabet_train'
test_dir = 'asl_alphabet_test'

batch_size = 32

# ImageDataGenerator for data augmentation and normalization 
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0, # Normalize the pixel values to be between 0 and 1
    shear_range=0.2, # Shear the image)
    zoom_range=0.2, # Zoom the image
    horizontal_flip=True, # Flip the image horizontally
    validation_split=0.2 # Split the data into training and validation sets
)

test_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Create training and validation generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(64, 64),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training' # Use the training subset
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(64, 64),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation' # Use the validation subset
)




Found 53 images belonging to 26 classes.
Found 0 images belonging to 26 classes.


In [ ]:
# Build the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')
])

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │       589,952 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 29)             │         3,741 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 686,941 (2.62 MB)

 Trainable params: 686,941 (2.62 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=20,
    verbose=1
)

Epoch 1/20
5577/5577 ━━━━━━━━━━━━━━━━━━━━ 573s 102ms/step - accuracy: 0.3563 - loss: 2.1264 - val_accuracy: 0.7050 - val_loss: 1.2806
Epoch 2/20
5577/5577 ━━━━━━━━━━━━━━━━━━━━ 0s 24us/step - accuracy: 0.6875 - loss: 0.7958 - val_accuracy: 0.6538 - val_loss: 1.4711
Epoch 3/20


c:\Users\awils\anaconda3\envs\dev4\lib\contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


5577/5577 ━━━━━━━━━━━━━━━━━━━━ 224s 40ms/step - accuracy: 0.7671 - loss: 0.6891 - val_accuracy: 0.7576 - val_loss: 1.2800
Epoch 4/20
5577/5577 ━━━━━━━━━━━━━━━━━━━━ 0s 9us/step - accuracy: 0.9375 - loss: 0.3129 - val_accuracy: 0.6923 - val_loss: 2.3403
Epoch 5/20
5577/5577 ━━━━━━━━━━━━━━━━━━━━ 223s 40ms/step - accuracy: 0.8402 - loss: 0.4763 - val_accuracy: 0.7797 - val_loss: 1.3965
Epoch 6/20
5577/5577 ━━━━━━━━━━━━━━━━━━━━ 0s 9us/step - accuracy: 0.9062 - loss: 0.2539 - val_accuracy: 0.7692 - val_loss: 1.5650
Epoch 7/20
5577/5577 ━━━━━━━━━━━━━━━━━━━━ 224s 40ms/step - accuracy: 0.8714 - loss: 0.3834 - val_accuracy: 0.7878 - val_loss: 1.4196
Epoch 8/20
5577/5577 ━━━━━━━━━━━━━━━━━━━━ 0s 6us/step - accuracy: 0.9375 - loss: 0.3829 - val_accuracy: 0.8846 - val_loss: 1.1090
Epoch 9/20
5577/5577 ━━━━━━━━━━━━━━━━━━━━ 224s 40ms/step - accuracy: 0.8914 - loss: 0.3243 - val_accuracy: 0.7983 - val_loss: 1.4526
Epoch 10/20
5577/5577 ━━━━━━━━━━━━━━━━━━━━ 0s 7us/step - accuracy: 0.8125 - loss: 0.3613 

In [ ]:
# Save the model
model.save('asl_model.keras')

In [ ]:
model = load_model('asl_model.keras')

# Define the function to predict a single image
def predict_single_image(img_path, model, labels):
    img = image.load_img(img_path, target_size=(64, 64))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_array)
    predicted_class_index = np.argmax(prediction, axis=1)
    predicted_label = labels[predicted_class_index[0]]
    
    return predicted_label

# Get the class labels
labels = (train_generator.class_indices)
labels = dict((v, k) for k, v in labels.items())

# Define the function to iterate through the folder and predict
def predict_folder(folder_path, model, labels):
    predictions = []
    for img_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_name)
        if os.path.isfile(img_path) and img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            predicted_label = predict_single_image(img_path, model, labels)
            predictions.append((img_name, predicted_label))
    return predictions



# Get predictions
predictions = predict_folder(test_dir, model, labels)

# Print predictions
for img_name, predicted_label in predictions:
    print(f'Image: {img_name}, Predicted Label: {predicted_label}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━